In [1]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
import random
import pickle
from tqdm import tqdm
import gc

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from keras import models, layers, losses, optimizers, regularizers, Model

2023-11-10 20:53:55.864418: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-10 20:53:55.864514: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-10 20:53:55.864624: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
gc.collect()

0

In [3]:
brain_index = pickle.load(open("support.pkl", "rb"))
nouns, verbs = pickle.load(open("vecs.pkl", "rb"))

pickles = [pickle.load(open(f"pickles/{i}.pkl", "rb")) for i in range(1)]
pickles = [item for sublist in pickles for item in sublist]
pickles = sorted(pickles, key=lambda x: x[1])
pickles = [[item for item in pickles if item[1] == noun] for noun in nouns]
pickles = [(np.add.reduce([item[0] for item in sublist]) / len(sublist), sublist[0][1]) for sublist in pickles]
pickles = [(item[0][brain_index], item[1]) for item in pickles]

In [4]:
class BasisSum(Model):
    def __init__(self):
        super().__init__()
        self.basis = tf.Variable(tf.convert_to_tensor([verbs[verb] for verb in verbs]), trainable=False)
        self.d1 = layers.Dense(256, activation="tanh")
        self.d2 = layers.Dense(self.basis.shape[0])

    def call(self, x):
        x = self.d1(x)
        x = self.d2(x)
        x = tf.einsum("bi, ij -> bj", x, self.basis)
        
        return x

In [5]:
def cos_sim(a, b):
    return dot(a, b) / (norm(a) * norm(b))

In [6]:
total = 500
pbar = tqdm(range(total))
correct_count = 0

for i in pbar:
    model = BasisSum()
    loss = losses.MeanSquaredError()
    opt = optimizers.Adam(0.01)

    random.shuffle(pickles)

    x = np.array([item[0] for item in pickles])
    y = [item[1] for item in pickles]
    y = np.array([nouns[item] for item in y])

    x, y = tf.cast(x, tf.dtypes.float32), tf.cast(y, tf.dtypes.float32)
    
    train_x, test_x = x[:-2], x[-2:]
    train_y, test_y = y[:-2], y[-2:]

    for j in range(1000):
        with tf.GradientTape() as tape:
            pred_y = model(train_x)
            batchloss = loss(train_y, pred_y)

            grad = tape.gradient(batchloss, model.trainable_variables)
            opt.apply_gradients(zip(grad, model.trainable_variables))


    pred = model(test_x)
    t1, t2 = test_y.numpy()
    t1, t2 = t1.flat, t2.flat
    p1, p2 = pred.numpy()
    p1, p2 = p1.flat, p2.flat
    
    correct = cos_sim(t1, p1) + cos_sim(t2, p2)
    incorrect = cos_sim(t1, p2) + cos_sim(t2, p1)

    correct_count += int(correct > incorrect)

    pbar.set_description(f"accuracy: {correct_count / (i + 1):.3f}, {correct}, {incorrect}")

accuracy: 0.529, 0.08863061666488647, 0.17334188520908356:  21%|██        | 104/500 [01:44<06:35,  1.00it/s]   